In [1]:
import os
import warnings
import sys

import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

import mlflow
import mlflow.sklearn

In [4]:
data_path = "wine-quality.csv"
data = pd.read_csv(data_path)

data.sample(10)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
3462,6.8,0.17,0.34,2.0,0.040,38.0,111.0,0.99000,3.24,0.45,12.900000,6
4141,6.8,0.28,0.29,11.9,0.052,51.0,149.0,0.99544,3.02,0.58,10.400000,6
2925,7.7,0.25,0.30,7.8,0.038,67.0,196.0,0.99555,3.10,0.50,10.100000,5
4626,6.3,0.30,0.91,8.2,0.034,50.0,199.0,0.99394,3.39,0.49,11.700000,6
4864,6.0,0.41,0.23,1.1,0.066,22.0,148.0,0.99266,3.30,0.47,9.633333,5
2142,7.5,0.26,0.52,13.2,0.047,64.0,179.0,0.99820,3.10,0.46,9.000000,5
13,6.6,0.16,0.40,1.5,0.044,48.0,143.0,0.99120,3.54,0.52,12.400000,7
3920,7.4,0.25,0.37,2.6,0.050,24.0,132.0,0.99138,3.04,0.53,11.200000,6
1786,7.4,0.33,0.26,2.6,0.040,29.0,115.0,0.99130,3.07,0.52,11.800000,7
968,6.5,0.08,0.33,1.9,0.028,23.0,93.0,0.99100,3.34,0.70,12.000000,7


In [5]:
# mlflow server --backend-store-uri mlruns/ --default-artifact-root mlruns/ --host 0.0.0.0 --port 5000
remote_server_uri = "http://0.0.0.0:5000" # set to your server URI
mlflow.set_tracking_uri(remote_server_uri)  # or set the MLFLOW_TRACKING_URI in the env

In [6]:
mlflow.tracking.get_tracking_uri()

'http://0.0.0.0:5000'

In [7]:
exp_name = "ElasticNet_wine"
mlflow.set_experiment(exp_name)

2024/04/08 12:37:26 INFO mlflow.tracking.fluent: Experiment with name 'ElasticNet_wine' does not exist. Creating a new experiment.


<Experiment: artifact_location='/home/jayashree/MLflow-practice/mlruns/449568220051209506', creation_time=1712560046538, experiment_id='449568220051209506', last_update_time=1712560046538, lifecycle_stage='active', name='ElasticNet_wine', tags={}>

In [10]:
def eval_metrics(actual, pred):
    # compute relevant metrics
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2


def load_data(data_path):
    data = pd.read_csv(data_path)

    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]
    return train_x, train_y, test_x, test_y


def train(alpha=0.5, l1_ratio=0.5):
    # train a model with given parameters
    warnings.filterwarnings("ignore")
    np.random.seed(40)

    # Read the wine-quality csv file (make sure you're running this from the root of MLflow!)
    data_path = "wine-quality.csv"
    train_x, train_y, test_x, test_y = load_data(data_path)

    # Useful for multiple runs (only doing one run in this sample notebook)    
    with mlflow.start_run():
        # Execute ElasticNet
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        # Evaluate Metrics
        predicted_qualities = lr.predict(test_x)
        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        # Print out metrics
        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        # Log parameter, metrics, and model to MLflow
        mlflow.log_param(key="alpha", value=alpha)
        mlflow.log_param(key="l1_ratio", value=l1_ratio)
        mlflow.log_metric(key="rmse", value=rmse)
        mlflow.log_metrics({"mae": mae, "r2": r2})
        mlflow.log_artifact(data_path)
        print("Save to: {}".format(mlflow.get_artifact_uri()))
        
        mlflow.sklearn.log_model(lr, "model")

In [11]:
train(0.5, 0.5)

Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.82224284975954
  MAE: 0.6278761410160691
  R2: 0.12678721972772689
Save to: /home/jayashree/MLflow-practice/mlruns/449568220051209506/7d934fac593b492fb467dd4d4db17786/artifacts


In [12]:
train(0.2, 0.2)

Elasticnet model (alpha=0.200000, l1_ratio=0.200000):
  RMSE: 0.7859129997062342
  MAE: 0.6155290394093895
  R2: 0.2022463182289208
Save to: /home/jayashree/MLflow-practice/mlruns/449568220051209506/9c099674167845f4b70aae81ba1a892f/artifacts


In [13]:
train(0.1, 0.1)

Elasticnet model (alpha=0.100000, l1_ratio=0.100000):
  RMSE: 0.7792546522251949
  MAE: 0.6112547988118587
  R2: 0.2157063843066196
Save to: /home/jayashree/MLflow-practice/mlruns/449568220051209506/fac7fb6b24794afc96bb389014dd4897/artifacts


In [15]:
from datetime import datetime
from mlflow.tracking import MlflowClient

client = MlflowClient()
experiments = client.search_experiments() # returns a list of mlflow.entities.Experiment
print(experiments)


[<Experiment: artifact_location='/home/jayashree/MLflow-practice/mlruns/449568220051209506', creation_time=1712560046538, experiment_id='449568220051209506', last_update_time=1712560046538, lifecycle_stage='active', name='ElasticNet_wine', tags={}>, <Experiment: artifact_location='/home/jayashree/MLflow-practice/mlruns/0', creation_time=1712559941248, experiment_id='0', last_update_time=1712559941248, lifecycle_stage='active', name='Default', tags={}>]


In [17]:
# get the run
_run = client.get_run(run_id="d88f3f5acc8241c99b0e79cc58fa778a")
print(_run)

<Run: data=<RunData: metrics={'mae': 0.6177463381374035,
 'r2': 0.19042414117747386,
 'rmse': 0.7917149365733894}, params={'alpha': '0.42', 'l1_ratio': '0.1'}, tags={'mlflow.gitRepoURL': 'git@github.com:JayashreeS11/MLflow-practice.git',
 'mlflow.log-model.history': '[{"run_id": "d88f3f5acc8241c99b0e79cc58fa778a", '
                             '"artifact_path": "model", "utc_time_created": '
                             '"2024-04-08 07:19:12.911862", "flavors": '
                             '{"python_function": {"model_path": "model.pkl", '
                             '"loader_module": "mlflow.sklearn", '
                             '"python_version": "3.6.8", "env": "conda.yaml"}, '
                             '"sklearn": {"pickled_model": "model.pkl", '
                             '"sklearn_version": "0.21.3", '
                             '"serialization_format": "cloudpickle"}}, '
                             '"model_uuid": '
                             '"1a2b48699a2b4fadb6

In [18]:
# add a tag to the run
dt = datetime.now().strftime("%d-%m-%Y (%H:%M:%S.%f)")
client.set_tag(_run.info.run_id, "deployed", dt)